In [1]:
import os
from google.cloud import bigquery
from google.cloud.bigquery import magics
from google.oauth2 import service_account
credentials = (service_account.Credentials.from_service_account_file(os.environ['GOOGLE_APPLICATION_CREDENTIALS']))
magics.context.credentials = credentials
magics.context.project = os.environ['GCP_PROJECT_ID']

In [2]:
%load_ext google.cloud.bigquery

In [3]:
client = bigquery.Client(project=os.environ['GCP_PROJECT_ID'])

#### Create dataset

In [4]:
client.create_dataset(f"{os.environ['GCP_PROJECT_ID']}.{os.environ['BQ_DATASET']}", exists_ok=True, timeout=30)

Dataset(DatasetReference('kaggle-notebook-to-production', 'customer_segmentation'))

#### Import data to dataset table.

In [7]:
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("InvoiceNo", "STRING"),
        bigquery.SchemaField("StockCode", "STRING"),
        bigquery.SchemaField("Description", "STRING"),
        bigquery.SchemaField("Quantity", "INTEGER"),
        bigquery.SchemaField("InvoiceDate", "STRING"),
        bigquery.SchemaField("UnitPrice", "FLOAT"),
        bigquery.SchemaField("CustomerID", "INTEGER"),
        bigquery.SchemaField("Country", "STRING")
],
    skip_leading_rows=1,
    source_format=bigquery.SourceFormat.CSV,
)
source_uri=f"gs://{os.environ['GCS_BUCKET_NAME']}/data.csv"
destination=f"{os.environ['GCP_PROJECT_ID']}.{os.environ['BQ_DATASET']}._raw"
load_job = client.load_table_from_uri(
    source_uri, destination, job_config=job_config
)

From origin data create table that makes PARSE_DATETIME(). Currently, in origin table, date is a string. Table name {raw}.

In [9]:
# CREATE OR REPLACE TABLE `kaggle-notebook-to-production.customer_segmentation.raw`  AS (
#   SELECT * EXCEPT(InvoiceDate), PARSE_DATETIME('%m/%d/%Y %H:%M', InvoiceDate) as InvoiceDate
#   FROM `kaggle-notebook-to-production.customer_segmentation._raw` 
# )

In [ ]:
# to check
# SELECT t1.InvoiceDate, t2.InvoiceDate 
# FROM `kaggle-notebook-to-production.customer_segmentation.raw` t1
# INNER JOIN `kaggle-notebook-to-production.customer_segmentation._raw` t2
# USING(InvoiceNo, StockCode, Quantity, CustomerID)

Check null/empty values. Run query with number of null values and relative number (%) null values for each column. Remove rows with empty value. Just query on  table {source}.

In [ ]:
# SELECT 
#   1 - COUNT(InvoiceNo)/COUNT(1) as InvoiceNo,
#   1 - COUNT(StockCode)/COUNT(1) as StockCode,
#   1 - COUNT(Description)/COUNT(1) as Description,
#   1 - COUNT(Quantity)/COUNT(1) as Quantity,
#   1 - COUNT(UnitPrice)/COUNT(1) as UnitPrice,
#   1 - COUNT(CustomerID)/COUNT(1) as CustomerID,
#   1 - COUNT(Country)/COUNT(1) as Country,
#   1 - COUNT(InvoiceDate)/COUNT(1) as InvoiceDate
# from 
# `kaggle-notebook-to-production.customer_segmentation.raw`


In [ ]:
# create or replace table `kaggle-notebook-to-production.customer_segmentation.source` as (
# select * from `kaggle-notebook-to-production.customer_segmentation.raw`
# where CustomerID is not null and Description is not null
# )

Build distribution by countries. Create table, that is ordered from most occurs country on the top. This table should include country name and relative number (%) from number of invoices (invoice and customer are a unique). Table name {by_countries}.

__Invoice IDs  are unique per CustomerID. It is not need to group by InvoiceId and CustomerID__

In [10]:
#select InvoiceNo, COUNT(DISTINCT CustomerID) as cn from `kaggle-notebook-to-production.customer_segmentation.source` 
#group by InvoiceNo having cn>0

In [ ]:
# SELECT Country, count(distinct InvoiceNo) cn FROM `kaggle-notebook-to-production.customer_segmentation.source` group by Country order by cn desc

Summary overview statistic for customers and products. Create query with number of unique customers, products and transactions. Create second table with number of product per invoice. Just query on  table {source}.

In [ ]:
# SELECT count(distinct InvoiceNo) transactions, count(distinct CustomerID) customers, count(distinct StockCode) products FROM `kaggle-notebook-to-production.customer_segmentation.source`


Create and check hypothesis that for each cancelled sale exist transaction sale. That sale row should have same  CustomerID, Description and positive  Quantity. Hypothesis 1. Just query on  table {source}.

In [ ]:
# select * from (
#     SELECT 
#     *
#     FROM `kaggle-notebook-to-production.customer_segmentation.source`
#     WHERE Quantity < 0
# ) t1
# left join `kaggle-notebook-to-production.customer_segmentation.source` t2 
# ON t1.CustomerID = t2.CustomerID and t1.Description=t2.Description and t1.Quantity = -t2.Quantity
# WHERE t2.CustomerID is null

Create and check hypothesis that each cancelled row have sale row with same StockCode and CustomerID and positive Quantity. For cancel rows Description filed should exclude word ‘Discount’. (Cancel rows should not have value ‘Discount’ in the Description column).  Just query on  table {source}.

In [ ]:
# SELECT 
# *
# FROM `kaggle-notebook-to-production.customer_segmentation.source`
# WHERE Quantity < 0
# ) t2
# WHERE t1.CustomerID = t2.CustomerID and t1.Description=t2.Description and t1.Quantity = -t2.Quantity
# group by  t1.CustomerID, t1.Description, t2.Quantity
# having cn > 1
# select count(1) from (
#     SELECT 
#     *
#     FROM `kaggle-notebook-to-production.customer_segmentation.source`
#     WHERE Quantity < 0 and Description != 'Discount'
# ) t1
# left join `kaggle-notebook-to-production.customer_segmentation.source` t2 
# ON t1.CustomerID = t2.CustomerID and t1.Description=t2.Description and t1.Quantity = -t2.Quantity
# WHERE t2.CustomerID is null

1. Create new field, called CancelQuantity (integer type). By default it filled by zero. Search all sale rows for each cancelled row. There are three cases:  Table name {source_v1}.
    1. No sale rows. Just remove cancelled row.
    2. One sale row. Set value in Quantity column from cancelled row to CancelQuantity in sale row. Remove cancelled row.
    3. More than one sale row. Set value from Quantity column in cancelled row to most recent sale row with Quantity value that is equal or greater than Quantity in cancelled row. Remove cancelled row.
    4. Remove all remaining rows with negative values in the Quantity column.

In [ ]:
# create or replace table  `kaggle-notebook-to-production.customer_segmentation.source_v1` as (
# with 
#     cancels as (
#         SELECT 
#         *
#         FROM `kaggle-notebook-to-production.customer_segmentation.source`
#         WHERE Quantity < 0 and Description != 'Discount'
#     ),

#     sales as (
#         SELECT 
#             *
#             FROM `kaggle-notebook-to-production.customer_segmentation.source`
#             WHERE Quantity >= 0

#     ),

#     join_cancels_sales as (
#         select s.*, c.Quantity CancelQuantity, c.InvoiceDate  CancelDate from sales s 
#         left join cancels c 
#         on s.CustomerID = c.CustomerID and s.Description=c.Description and s.StockCode=c.StockCode
#     ),
#     filtered_by_date as (
#         select * except (CancelQuantity, CancelDate), 
#         IF( CancelDate<InvoiceDate, Null, CancelQuantity) CancelQuantity,
#         IF( CancelDate<InvoiceDate, Null, CancelDate) CancelDate   
#         from join_cancels_sales
#     ),
#     filtered_by_qty as (
#         select * except (CancelQuantity, CancelDate), 
#         IF( Quantity<-1*CancelQuantity, Null, CancelQuantity) CancelQuantity,
#         IF( Quantity<-1*CancelQuantity, Null, CancelDate) CancelDate   
#         from filtered_by_date
#     ), 
#     with_number_in_multiple_sales as (
#         select *, ROW_NUMBER() OVER (
#             PARTITION BY StockCode, Description, CustomerID, CancelQuantity, CancelDate
#             ORDER BY InvoiceDate DESC
#         ) AS row_num from filtered_by_date
#     ),
#     filter_by_multiple_sales as (
#         select * except (CancelQuantity, CancelDate), 
#         IF( row_num > 1, Null, CancelQuantity) CancelQuantity,
#         IF( row_num > 1, Null, CancelDate) CancelDate   
#         from with_number_in_multiple_sales
#     )
#     select * except (CancelDate, row_num, CancelQuantity), IFNULL(CancelQuantity, 0) CancelQuantity  from filter_by_multiple_sales 
# )

#### The previous query finished with failuer. It need more time to re-produce transforming cancelled transactions into column. Currentlly, I download and use verioson from origin and load it to BQ table with name `kaggle-notebook-to-production:customer_segmentation.source_v1_from_origin`. The CSV file with data is stored here: `gs://kntp-customer-segmentation/df_cleaned.csv`


Create new column TotalPrice. This column is computed as (Quantity - CancelQuantity)  multiplied by UnitPrice. Table name {source_v2}.

In [3]:
# create or replace table `kaggle-notebook-to-production.customer_segmentation.source_v2` as (
#     SELECT 
# * except (int64_field_0),((Quantity+QuantityCanceled) * UnitPrice) TotalPrice

# FROM `kaggle-notebook-to-production.customer_segmentation.source_v1_from_origin`
# )

Create new column that includes squashes (grouped) rows belonged to same InvoiceNo. The InvoiceDate should include mean values from all InvoiceDates in the one transaction. Column Basket Price equals to sum of all UnitPrice in one invoice. PS: group by Customer and Invoice. Also remove rows (that are represent invoices) with BasketPrice == 0. Table name {source_v3}.

In [ ]:
# create or replace table `kaggle-notebook-to-production.customer_segmentation.source_v3` as (
#     SELECT 
#     InvoiceNo, 
#     CustomerID, 
#     DATETIME(TIMESTAMP_SECONDS(cast (avg(UNIX_SECONDS(TIMESTAMP(InvoiceDate))) as INT64))) as InvoiceDate,
#     round(sum(TotalPrice), 2) BasketPrice
#     from `kaggle-notebook-to-production.customer_segmentation.source_v2`
#     group by InvoiceNo, CustomerID
#     having BasketPrice > 0
# )
# select DATETIME(TIMESTAMP_SECONDS(cast (avg(UNIX_SECONDS(TIMESTAMP(InvoiceDate))) as INT64))) as c from `kaggle-notebook-to-production.customer_segmentation.source_v2`
# where InvoiceDate > '2011-01-01' and InvoiceDate <= '2011-01-31'

Create table that  bins BasketPrice in such groups: [0, 50, 100, 200, 500, 1000, 5000, 50000] and compute the count. For example number of Invoices, placed between $0-$50 is XXX. Create table with two columns: Partition (Bin name) and count invoices. Table name {source_v4}.

In [ ]:
# -- [min, max)
# DECLARE NUM_ALL_ROWS INT64;
# SET NUM_ALL_ROWS = (SELECT count(1) from `kaggle-notebook-to-production.customer_segmentation.source_v3`);

# select bin, concat(cast(round(cn/NUM_ALL_ROWS * 100, 1) as STRING), "%") from (
#     SELECT "[0, 50)" bin,  count(1) cn FROM `kaggle-notebook-to-production.customer_segmentation.source_v3` 
#     WHERE BasketPrice >= 0 and BasketPrice < 50
#     UNION ALL 
#     SELECT "[50, 100)" bin,  count(1) cn FROM `kaggle-notebook-to-production.customer_segmentation.source_v3` 
#     WHERE BasketPrice >= 50 and BasketPrice < 100
#     UNION ALL 
#     SELECT "[100, 200)" bin,  count(1) cn FROM `kaggle-notebook-to-production.customer_segmentation.source_v3` 
#     WHERE BasketPrice >= 100 and BasketPrice < 200
#     UNION ALL 
#     SELECT "[200, 500)" bin,  count(1) cn FROM `kaggle-notebook-to-production.customer_segmentation.source_v3` 
#     WHERE BasketPrice >= 200 and BasketPrice < 500
#     UNION ALL 
#     SELECT "[500, 1000)" bin,  count(1) cn FROM `kaggle-notebook-to-production.customer_segmentation.source_v3` 
#     WHERE BasketPrice >= 500 and BasketPrice < 1000
#     UNION ALL 
#     SELECT "[1000, 5000)" bin,  count(1) cn FROM `kaggle-notebook-to-production.customer_segmentation.source_v3` 
#     WHERE BasketPrice >= 1000 and BasketPrice < 5000
#     UNION ALL 
#     SELECT "[5000, 50000)" bin,  count(1) cn FROM `kaggle-notebook-to-production.customer_segmentation.source_v3` 
#     WHERE BasketPrice >= 5000 and BasketPrice < 50000
# ) order by cn desc 

1. Create categories for the product. Using descriptions of the product. This step include tree technics in  NLP: word tokenizing, steaming and pos_tagging. Steps:  
    1. Create table with unique descriptions (maybe add Stock code for matching). Table name {descriptions}.
    2. Iterate over each word in the description (word tokenizer), filter by noun (pos tagging) and create steam (steaming). Create two maps. Root keywords: keys is the steam of the each word and values are the actual words. Each key should include one or more values. Second map Count steams: key is the steam and values is the number of occurrences of the each keyword. 
    3. Create map (clean root keywords) with steam as key and one word as the value. In case, map root keywords include more than one words - select one with the smallest size.
    4. Remain only steams (in clean root keywords map) that are occurred more that 13 times.

__This previous steps (NLP steps) I did not implement in BigQuery. From three technics are needed for NLP tasks: word tokenizing, steaming and pos_tagging BigQuery has only tokenizing. Thus, I import dataset from origin (pandas) approach. I imported X dataset, that includes one-hot-encoding for each description usign words. Stored in the `kaggle-notebook-to-production.customer_segmentation.description_ohe` from CSV file: `gs://kntp-customer-segmentation/ohe-descriptions_normalized.csv`__ 

__In the following code, I added prefix `_` to all colunm that represented the word.I do so, because there is an error, cause column has non appropriate (for BQ) name.__

In [4]:
import pandas as pd
df = pd.read_csv('data/ohe-descriptions.csv')

In [10]:
df = df.rename(columns={c:f"_{c}" for c in df.columns if c not in ['Description', 'Unnamed: 0']}).drop(columns=['Unnamed: 0'])
df.to_csv('data/ohe-descriptions_normalized.csv')

Create table (matrix) as a feature table for build cluster K-Means model. Columns in this table are worlds (one-hot-encoding) and plus 6 addition columns. Last 6 column are representing number of bin for UnitPrice. There are following bins: [0, 1, 2, 3, 5, 10]. Table name {features_descriptions}.

In [1]:
# create or replace table `kaggle-notebook-to-production.customer_segmentation.features_description` as (
#     with 
#     mean_price_desc as (
#         SELECT Description, round(avg(UnitPrice),2) price  FROM `kaggle-notebook-to-production.customer_segmentation.source_v2` group by Description
#     ),
#     ohe_mean_price_desc as (
#         select 
#         Description,
#         price,
#         IF(price>=0 AND price < 1, 1, 0)  s0_f1,
#         IF(price>=1 AND price < 2, 1, 0)  s1_f2,
#         IF(price>=2 AND price < 3, 1, 0)  s2_f3,
#         IF(price>=3 AND price < 5, 1, 0)  s3_f5,
#         IF(price>=5 AND price < 10, 1, 0)  s5_f10,
#         IF(price>=10, 1, 0)  s10_f_
#         from mean_price_desc 
#         group by Description, price
#     )
#     select t1.* except (int64_field_0),
#     t2.* except (Description) from `kaggle-notebook-to-production.customer_segmentation.description_ohe` t1
#     inner join ohe_mean_price_desc t2 using(Description) 

# )

Apply k-means algorithm with 5 centroids to build the model, that can categories description.

In [ ]:
# CREATE MODEL
#   `kaggle-notebook-to-production.customer_segmentation.cat_desc_5_cent`
# OPTIONS
#   ( MODEL_TYPE='KMEANS',
#     KMEANS_INIT_METHOD='KMEANS++',
#     NUM_CLUSTERS=5 ) AS
# SELECT
#   * except (Description, price)
# FROM `kaggle-notebook-to-production.customer_segmentation.features_description`

__Important: BigQuery approach to create kmeans model does not create good model. That model has very imbalanced clusters. I checked the feature between pandas and BigQuery, they are same. I used the same options: init algorithm (kmeam++), number of clusters (5). But algoritm converged after 3-5 iterations. Problem: model provide cluster with 98% of all data and other 2% are spread between remaind 4 clusters. Table name: `kaggle-notebook-to-production.customer_segmentation.description_clusters`. Source file: `gs://kntp-customer-segmentation/description_clusters.csv`__


Create new table from origin (before grouping by InvoiceNo) and add 6 new columns: one-hot-encode for each of 5 categories and one that represent number of the category. But instead of filling zeros and ones categorical columns, fill it with TotalPrice.

In [ ]:
# create or replace table `kaggle-notebook-to-production.customer_segmentation.source_v4` as (
# select 
#     t1.*,
#     IF(t2.cluster = 1, t1.TotalPrice, 0)  categ_0,
#     IF(t2.cluster = 2, t1.TotalPrice, 0)  categ_1,
#     IF(t2.cluster = 3, t1.TotalPrice, 0)  categ_2,
#     IF(t2.cluster = 4, t1.TotalPrice, 0)  categ_3,
#     IF(t2.cluster = 5, t1.TotalPrice, 0)  categ_4,
#     t2.cluster categ_product
# from 
#     `kaggle-notebook-to-production.customer_segmentation.source_v2` t1
# left join  `kaggle-notebook-to-production.customer_segmentation.description_clusters` t2 using (Description)
# )

Create new table that include squashes  (grouped) rows from previous table by InvoiceNo. Each row represent each separate Invoice. Sum  categories, BasketPrice. Average the InvoiceDate. Table name {basket_price}.

In [ ]:
# create or replace table `kaggle-notebook-to-production.customer_segmentation.basket_price` as (
#     SELECT 
#     InvoiceNo, 
#     CustomerID, 
#     DATETIME(TIMESTAMP_SECONDS(cast (avg(UNIX_SECONDS(TIMESTAMP(InvoiceDate))) as INT64))) as InvoiceDate,
#     round(sum(TotalPrice), 2) BasketPrice,
#     round(sum(categ_0), 2) categ_0,
#     round(sum(categ_1), 2) categ_1,
#     round(sum(categ_2), 2) categ_2,
#     round(sum(categ_3), 2) categ_3,
#     round(sum(categ_4), 2) categ_4,
#     from `kaggle-notebook-to-production.customer_segmentation.source_v4`
#     group by InvoiceNo, CustomerID
#     having BasketPrice > 0
# )

Split data for train and test. Use last 2 months as test dataset and remaining data for train stage. During building  customer categories and model for predicting cluster using only train dataset.  Table names {train_basket_price/test_basket_price}.

In [ ]:
# create or replace table `kaggle-notebook-to-production.customer_segmentation.train_basket_price` as (
#     select * from `kaggle-notebook-to-production.customer_segmentation.basket_price` where InvoiceDate <= '2011-10-01'
# );
# create or replace table `kaggle-notebook-to-production.customer_segmentation.test_basket_price` as (
#     select * from `kaggle-notebook-to-production.customer_segmentation.basket_price` where InvoiceDate > '2011-10-01'
# );
#
# Debug splitting.
# SELECT 'train' as t, max(InvoiceDate), min(InvoiceDate) FROM `kaggle-notebook-to-production.customer_segmentation.train_basket_price`
# union all 
# SELECT 'test' as t, max(InvoiceDate), min(InvoiceDate) FROM `kaggle-notebook-to-production.customer_segmentation.test_basket_price`


Create new table and squash table from 9 (squashed by Invoice) by CustomerId. Apply sum, count, min, max, mean for BasketPrice column. It spawns 5 columns. For categorical columns (5 columns) compute relative value. Sum of each category for CustomerId divided by sum of all BasketPrice multiply by 100. Add two columns that are represented number of days from first and last InvoiceDate (depend of the last day in the dataset). Table name {train_transactions_per_user}.

In [ ]:
# DECLARE LAST_DATE DATETIME;
# SET LAST_DATE = (SELECT max(InvoiceDate) from `kaggle-notebook-to-production.customer_segmentation.train_basket_price`);

# create or replace table `kaggle-notebook-to-production.customer_segmentation.train_transactions_per_user` as (
#     select 
#         CustomerID,
#         count(1) _count,
#         min(BasketPrice) _min,
#         max(BasketPrice) _max,
#         round(avg(BasketPrice), 2) _avg,
#         round(sum(BasketPrice), 2) _sum,
#         sum(categ_0)/sum(BasketPrice)*100 categ_0,
#         sum(categ_1)/sum(BasketPrice)*100 categ_1,
#         sum(categ_2)/sum(BasketPrice)*100 categ_2,
#         sum(categ_3)/sum(BasketPrice)*100 categ_3,
#         sum(categ_4)/sum(BasketPrice)*100 categ_4,
#         date_diff(LAST_DATE, min(InvoiceDate),DAY) as FirstPurchase,
#         date_diff(LAST_DATE, max(InvoiceDate),DAY) as LastPurchase
        
#     from 
#         `kaggle-notebook-to-production.customer_segmentation.train_basket_price`
#     group by CustomerID
# )


In [ ]:
# create or replace model  `kaggle-notebook-to-production.customer_segmentation.trs_user_model_c11` 
# TRANSFORM (
#     ML.STANDARD_SCALER(_count) OVER() _count,
#     ML.STANDARD_SCALER(_min) OVER() _min,
#     ML.STANDARD_SCALER(_max) OVER() _max,
#     ML.STANDARD_SCALER(_avg) OVER() _avg,
#     ML.STANDARD_SCALER(categ_0) OVER() categ_0,
#     ML.STANDARD_SCALER(categ_1) OVER() categ_1,
#     ML.STANDARD_SCALER(categ_2) OVER() categ_2,
#     ML.STANDARD_SCALER(categ_3) OVER() categ_3,
#     ML.STANDARD_SCALER(categ_4) OVER() categ_4
# )
# OPTIONS (
#     MODEL_TYPE='KMEANS',
#     KMEANS_INIT_METHOD='KMEANS++',
#     NUM_CLUSTERS=11
# )
# as
# SELECT * from `kaggle-notebook-to-production.customer_segmentation.train_transactions_per_user`


In [ ]:
# select CENTROID_ID, count(1) cn from 
# ML.PREDICT(MODEL `kaggle-notebook-to-production.customer_segmentation.trs_user_model_c11` ,
#  TABLE `kaggle-notebook-to-production.customer_segmentation.train_transactions_per_user`)
#  group by CENTROID_ID
#  order by cn desc 


Apply k-means algorithm with number of centroid equals 11. Store all fields from train_transactions_per_user and add addition one field, called, label to  Table name {labels_transactions_per_user}.

In [ ]:
# create or replace table `kaggle-notebook-to-production.customer_segmentation.labels_transactions_per_user` as (
#     select 
#         * except (NEAREST_CENTROIDS_DISTANCE, CENTROID_ID), CENTROID_ID label 
#     from 
#         ML.PREDICT(MODEL `kaggle-notebook-to-production.customer_segmentation.trs_user_model_c11` ,
#             TABLE `kaggle-notebook-to-production.customer_segmentation.train_transactions_per_user`)
# )

1. Create model for predicting cluster (category) for users with first (one) transaction (invoice):
    1. Create feature set with ['mean', 'categ_0', 'categ_1', 'categ_2', 'categ_3', 'categ_4’ ]. Use full previous dataset (that used during building k-mean). Using Table {labels_transactions_per_user}.
    2. Split to train and validation. Doing this in the SQL query.
    3. Using different algorithm with grid search and cross validation number folds=5. Use: Logistic regression, XGBoost, SVM. Author applied VotingClassifier.

XGBoost

In [ ]:
# CREATE MODEL `kaggle-notebook-to-production.customer_segmentation.cat_user_xgb`
# OPTIONS(MODEL_TYPE='BOOSTED_TREE_CLASSIFIER',
#         BOOSTER_TYPE = 'GBTREE',
#         NUM_PARALLEL_TREE = 1,
#         MAX_ITERATIONS = 40,
#         TREE_METHOD = 'HIST',
#         EARLY_STOP = FALSE,
#         SUBSAMPLE = 0.8,
#         INPUT_LABEL_COLS = ['label'])
# AS SELECT 
#     _avg, categ_0,categ_1,categ_2,categ_3,categ_4, label
#  FROM `kaggle-notebook-to-production.customer_segmentation.labels_transactions_per_user`

Logistic regression

In [ ]:
# CREATE MODEL
#   `kaggle-notebook-to-production.customer_segmentation.cat_user_lr`
# OPTIONS
#   ( MODEL_TYPE='LOGISTIC_REG',
#     AUTO_CLASS_WEIGHTS=TRUE,
#     SUBSAMPLE = 0.8,
#     INPUT_LABEL_COLS = ['label']
#      ) AS
# SELECT 
#     _avg, categ_0,categ_1,categ_2,categ_3,categ_4, label
#  FROM `kaggle-notebook-to-production.customer_segmentation.labels_transactions_per_user`